# Question A2 (10 marks)

### In this question, we will determine the optimal batch size for mini-batch gradient descent. Find the optimal batch size for mini-batch gradient descent by training the neural network and evaluating the performances for different batch sizes. Note: Use 5-fold cross-validation on the training partition to perform hyperparameter selection. You will have to reconsider the scaling of the dataset during the 5-fold cross validation.

#### Plot mean cross-validation accuracies on the final epoch for different batch sizes as a scatter plot. Limit search space to batch sizes {64, 128, 256, 512}. Next, create a table of time taken to train the network on the last epoch against different batch sizes. Finally, select the optimal batch size and state a reason for your selection.


This might take a while to run, so plan your time carefully.

In [1]:
import tqdm
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from common_utils import set_seed

# setting seed
set_seed()

1.To reduce repeated code, place your

- network (MLP defined in QA1)

- torch datasets (CustomDataset defined in QA1)
- loss function (loss_fn defined in QA1)
in a separate file called common_utils.py

Import them into this file. You will not be repenalised for any error in QA1 here as the code in QA1 will not be remarked.

The following code cell will not be marked.

In [ ]:
# YOUR CODE HERE
from common_utils import split_dataset, preprocess_dataset, MLP, CustomDataset, preprocess

2.Define different folds for different batch sizes to get a dictionary of training and validation datasets. Preprocess your datasets accordingly.

In [ ]:
def generate_cv_folds_for_batch_sizes(parameters, X_train, y_train):
    """
    returns:
    X_train_scaled_dict(dict) where X_train_scaled_dict[batch_size] is a list of the preprocessed training matrix for the different folds.
    X_val_scaled_dict(dict) where X_val_scaled_dict[batch_size] is a list of the processed validation matrix for the different folds.
    y_train_dict(dict) where y_train_dict[batch_size] is a list of labels for the different folds
    y_val_dict(dict) where y_val_dict[batch_size] is a list of labels for the different folds
    """
    X_train_scaled_dict = {}
    X_val_scaled_dict = {}
    y_train_dict = {}
    y_val_dict = {}

    # Create a KFold object for 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Iterate over batch sizes
    for batch_size in batch_sizes:
        X_train_scaled_dict[batch_size] = []
        X_val_scaled_dict[batch_size] = []
        y_train_dict[batch_size] = []
        y_val_dict[batch_size] = []

        # Iterate over cross-validation folds
        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
            
            # Scale the features
            X_train_scaled_fold, X_val_scaled_fold = preprocess_dataset(X_train_fold, X_val_fold)

            # Store the scaled data and labels
            X_train_scaled_dict[batch_size].append(X_train_scaled_fold)
            X_val_scaled_dict[batch_size].append(X_val_scaled_fold)
            y_train_dict[batch_size].append(y_train_fold)
            y_val_dict[batch_size].append(y_val_fold)

    return X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict

batch_sizes = [64, 128, 256, 512]
X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict = generate_cv_folds_for_batch_sizes(batch_sizes, X_train.to_numpy(), y_train)


3.Perform hyperparameter tuning for the different batch sizes with 5-fold cross validation.

In [ ]:
from common_utils import EarlyStopper

epochs = 100
early_stopper = EarlyStopper(patience=3, min_delta=0)

def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        correct += ((pred > 0.5).type(torch.float) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(correct*100):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    
    return train_loss, correct

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += ((pred > 0.5).type(torch.float) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_loss, correct

def find_optimal_hyperparameter(X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict, parameters, parameter_name):
    """
    returns:
    cross_validation_accuracies(list) where cross_validation_accuracies[i] is the cross validation accuracy for the ith parameter value
    cross_validation_times(list) where cross_validation_times[i] is the mean time taken on the last epoch for the ith parameter value
    """

    cross_validation_accuracies = []
    cross_validation_times = []

    for parameter in parameters:
        print(f"Training for {parameter_name} = {parameter}")
        accuracies = []
        times = []

        for i in range(5):
            print(f"Fold {i+1}")
            model = MLP(77,128,1)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            loss_fn = nn.BCELoss()
            early_stopper = EarlyStopper(patience=3, min_delta=0)

            train_data = CustomDataset(X_train_scaled_dict[parameter][i], y_train_dict[parameter][i])
            val_data = CustomDataset(X_val_scaled_dict[parameter][i], y_val_dict[parameter][i])

            train_dataloader = DataLoader(train_data, batch_size=parameter, shuffle=True)
            val_dataloader = DataLoader(val_data, batch_size=parameter, shuffle=True)

            train_loss, test_loss = [], []
            train_acc, test_acc = [], []
            epoch_times = []
            
            for t in range(epochs):
                
                print(f"Epoch {t+1}\n-------------------------------")

                # Training Loop
                start = time.time()
                tr_loss, acc = train_loop(train_dataloader, model, loss_fn, optimizer)
                train_loss.append(tr_loss), train_acc.append(acc)

                # Validation Loop
                te_loss, acc = test_loop(val_dataloader, model, loss_fn)
                test_loss.append(te_loss), test_acc.append(acc)
                end = time.time()

                epoch_times.append(end-start)

                if early_stopper.early_stop(te_loss): 
                    print("Done!")                    
                    break
            
            # Accuracy for the current Fold at last Epoch
            accuracies.append(test_acc[-1])

            # Time taken for the current Fold at last Epoch
            times.append(epoch_times[-1])

        # Mean Accuracy for the Batch Size (Mean of Accuracy at Last Epoch for each Fold)
        cross_validation_accuracies.append(np.mean(accuracies))

        # Mean Time taken for the Batch Size (Mean of Time at Last Epoch for each Fold)
        cross_validation_times.append(np.mean(times))

    return cross_validation_accuracies, cross_validation_times

batch_sizes = [64, 128, 256, 512]
cross_validation_accuracies, cross_validation_times = find_optimal_hyperparameter(X_train_scaled_dict, X_val_scaled_dict, y_train_dict, y_val_dict, batch_sizes, 'batch_size')


4.Plot scatterplot of mean cross validation accuracies for the different batch sizes.

In [2]:
# YOUR CODE HERE
def plot_cross_validation_accuracies(cross_validation_accuracies, parameters, parameter_name):
    plt.plot(parameters, cross_validation_accuracies, marker='x',linestyle='None')
    plt.xlabel(parameter_name)
    plt.ylabel('Cross Validation Accuracy')
    plt.show()

plot_cross_validation_accuracies(cross_validation_accuracies, batch_sizes, 'batch_size')

4.Create a table of time taken to train the network on the last epoch against different batch sizes. Select the optimal batch size and state a reason for your selection.

In [ ]:
df = pd.DataFrame({'Batch Size': batch_sizes,
                   'Last Epoch Time': cross_validation_times   
                  })

df

In [ ]:
# YOUR CODE HERE
optimal_batch_size =
reason =  ""